**Modelos de Regresión Personalizados**

Diseñar y optimizar modelos de regresión mediante la búsqueda sistemática de hiperparámetros. Se debe incluir un análisis de residuos y gráficos de dispersión que comparen los valores reales con las predicciones. Justificar las decisiones tomadas en la configuración de los hiperparámetros con base en analisis coherentes de los datos.

In [ ]:
%run './02-analisis-calidad-datos-cashrequest.ipynb'
df_cashrequest = df_cashrequest.copy()  # Guardar el primer df

df_fees = pd.read_csv('./fees_new.csv').copy()  # Guardar el segundo df

In [114]:
import sklearn.model_selection as train_test_split
import sklearn.svm as svm

In [115]:
df_cashrequest.tail(10)

,id,amount,status,created_at,updated_at,user_id,moderated_at,deleted_account_id,reimbursement_date,cash_request_received_date,...,send_at,recovery_status,reco_creation,reco_last_update,time_to_update,transaction_month,cohort,payment_issue,recovery_status_encoded,transfer_type_encoded
23960,26969,50.0,direct_debit_sent,2020-11-01 17:31:03.345533+00:00,2021-01-06 09:07:31.575691+00:00,3377.0,NaT,NaN,NaT,2020-11-02,...,2020-11-01 17:31:25.526137+00:00,NaN,NaT,NaT,1575.607842,2020-11,2020-11,False,4,1
23961,18579,100.0,money_back,2020-09-30 14:28:32.645872+00:00,2021-02-06 20:16:09.511010+00:00,57130.0,2020-09-30 17:09:57.946096+00:00,NaN,NaT,2020-10-01,...,2020-10-07 14:28:32.601819+00:00,NaN,NaT,NaT,3101.793574,2020-09,2020-09,False,4,1
23962,26979,100.0,money_back,2020-11-01 19:08:24.111371+00:00,2021-02-05 20:48:23.270789+00:00,102737.0,NaT,NaN,NaT,NaT,...,2020-11-01 19:08:48.445188+00:00,NaN,NaT,NaT,2305.666433,2020-11,2020-11,False,4,1
23963,7930,100.0,money_back,2020-07-14 08:55:09.360786+00:00,2021-02-05 12:04:06.194796+00:00,31413.0,2020-07-14 12:00:53.394529+00:00,NaN,NaT,2020-07-16,...,NaT,completed,2020-08-11 22:25:55.349356+00:00,2021-02-05 12:04:06.147841+00:00,4947.149121,2020-07,2020-07,False,0,0
23964,16834,100.0,money_back,2020-09-19 07:37:14.351567+00:00,2021-02-05 12:11:58.186458+00:00,20064.0,2020-09-19 15:37:54.670099+00:00,NaN,NaT,2020-09-22,...,2020-09-26 07:37:14.299162+00:00,completed,2020-10-15 22:27:12.949488+00:00,2021-02-05 12:11:58.134306+00:00,3340.578843,2020-09,2020-09,False,0,1
23965,20616,100.0,money_back,2020-10-12 13:54:11.686225+00:00,2021-02-06 20:17:49.292493+00:00,13681.0,NaT,NaN,NaT,2020-10-13,...,2020-10-12 13:54:24.352856+00:00,NaN,NaT,NaT,2814.393780,2020-10,2020-10,False,4,1
23966,25243,50.0,money_back,2020-10-27 14:41:25.734910+00:00,2020-12-18 13:15:40.843946+00:00,NaN,NaT,30367.0,NaT,2020-10-28,...,2020-10-27 14:41:57.901946+00:00,completed,2020-11-12 23:20:41.928788+00:00,2020-12-01 13:26:53.815504+00:00,1246.570864,2020-10,2020-10,False,0,1
23967,22357,100.0,money_back,2020-10-20 07:58:04.006937+00:00,2021-02-05 12:19:30.656816+00:00,82122.0,NaT,NaN,NaT,2020-10-21,...,2020-10-20 07:58:14.171553+00:00,NaN,NaT,NaT,2596.357403,2020-10,2020-10,False,4,1
23968,20256,100.0,money_back,2020-10-10 05:40:55.700422+00:00,2021-02-05 13:14:19.707627+00:00,64517.0,NaT,NaN,NaT,2020-10-12,...,2020-10-10 05:41:23.368363+00:00,NaN,NaT,NaT,2839.556669,2020-10,2020-10,False,4,1
23969,19886,100.0,direct_debit_sent,2020-10-08 14:16:52.155661+00:00,2021-01-05 15:45:52.645536+00:00,44867.0,NaT,NaN,NaT,2020-10-10,...,2020-10-08 14:17:04.526139+00:00,NaN,NaT,NaT,2137.483469,2020-10,2020-10,False,4,1


Sabemos que:
recovery_status_mapping: 
    'completed': 0,
    'pending': 1,
    'pending_direct_debit': 2,
    'cancelled': 3,
    np.nan: 4 

transfer_type_mapping: 
    'regular': 0,
    'instant': 1

In [ ]:
df_cashrequest['reimbursement_month'] = df_cashrequest['reimbursement_date'].dt.month
df_cashrequest['created_at_month'] = df_cashrequest['created_at'].dt.month
df_cashrequest['updated_at_month'] = df_cashrequest['updated_at'].dt.month


In [ ]:
df_cashrequest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23970 entries, 0 to 23969
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   id                          23970 non-null  int64              
 1   amount                      23970 non-null  float64            
 2   status                      23970 non-null  object             
 3   created_at                  23970 non-null  datetime64[ns, UTC]
 4   updated_at                  23970 non-null  datetime64[ns, UTC]
 5   user_id                     21867 non-null  float64            
 6   moderated_at                15912 non-null  datetime64[ns, UTC]
 7   deleted_account_id          2104 non-null   float64            
 8   reimbursement_date          3050 non-null   datetime64[ns, UTC]
 9   cash_request_received_date  16289 non-null  datetime64[ns]     
 10  money_back_date             12040 non-null  datetime64[ns,

In [ ]:
#Crear df numérico
# Selecting only numeric columns from the dataset
numeric_columns = df_cashrequest.select_dtypes(include=['int64', 'float64']).columns

# Creating a new DataFrame with only numeric columns
cashrequest = df_cashrequest[numeric_columns]

In [ ]:
cashrequest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23970 entries, 0 to 23969
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       23970 non-null  int64  
 1   amount                   23970 non-null  float64
 2   user_id                  21867 non-null  float64
 3   deleted_account_id       2104 non-null   float64
 4   time_to_update           23970 non-null  float64
 5   recovery_status_encoded  23970 non-null  int64  
 6   transfer_type_encoded    23970 non-null  int64  
 7   status_encoded           23970 non-null  int64  
 8   reimbursement_month      3050 non-null   float64
 9   payment_issue_encoded    23970 non-null  int64  
dtypes: float64(5), int64(5)
memory usage: 1.8 MB


In [116]:
df_fees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21061 entries, 0 to 21060
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   id                21061 non-null  int64              
 1   cash_request_id   21057 non-null  float64            
 2   type              21061 non-null  object             
 3   status            21061 non-null  object             
 4   category          2196 non-null   object             
 5   total_amount      21061 non-null  float64            
 6   reason            21061 non-null  object             
 7   created_at        21061 non-null  datetime64[ns, UTC]
 8   updated_at        21061 non-null  datetime64[ns, UTC]
 9   paid_at           15438 non-null  datetime64[ns, UTC]
 10  from_date         6749 non-null   datetime64[ns, UTC]
 11  to_date           6512 non-null   datetime64[ns, UTC]
 12  charge_moment     21061 non-null  object             
 13  c

In [ ]:
df_fees.nunique()

id                 21061
cash_request_id    12933
type                   3
status                 4
category               2
total_amount           2
reason             15149
created_at         21026
updated_at         21061
paid_at            15438
from_date            454
to_date              486
charge_moment          2
cohort                 7
dtype: int64

In [113]:
df_fees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21061 entries, 0 to 21060
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   id                21061 non-null  int64              
 1   cash_request_id   21057 non-null  float64            
 2   type              21061 non-null  object             
 3   status            21061 non-null  object             
 4   category          2196 non-null   object             
 5   total_amount      21061 non-null  float64            
 6   reason            21061 non-null  object             
 7   created_at        21061 non-null  datetime64[ns, UTC]
 8   updated_at        21061 non-null  datetime64[ns, UTC]
 9   paid_at           15438 non-null  datetime64[ns, UTC]
 10  from_date         6749 non-null   datetime64[ns, UTC]
 11  to_date           6512 non-null   datetime64[ns, UTC]
 12  charge_moment     21061 non-null  object             
 13  c